# Q-learning algorithm

[Train an AI to solve the Frozen Lake environment](https://mlabonne.github.io/blog/reinforcement%20learning/q-learning/frozen%20lake/gym/tutorial/2022/02/13/Q_learning.html)


In [1]:
import gymgrid # pip install gymgrid
import numpy as np
import gym

gym.__version__

'0.26.2'

In [2]:
environment = gym.make("FrozenLake-v1", is_slippery=False)
environment.reset()  # (0, {'prob': 1})
environment.render()  # warning: "render mode"

# You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")

# So I do what it says, and I get an error #:(
# gym("FrozenLake-v1", render_mode="rgb_array")  # Error: "module object is not callable"


/usr/local/anaconda3/lib/python3.9/site-packages/gym/envs/toy_text/frozen_lake.py:271: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")
  logger.warn(


## Reinforcement Learning

In [3]:
# 16 positions, states.
# 4 actions; left, right, up, down.
# Our table has the following dimensions:
# (rows x columns) = (states x actions) = (16 x 4)
qtable = np.zeros((16, 4))

# Alternatively, the gym library can also directly g
# give us the number of states and actions using 
# "env.observation_space.n" and "env.action_space.n"
nb_states = environment.observation_space.n  # = 16
nb_actions = environment.action_space.n      # = 4
qtable = np.zeros((nb_states, nb_actions))

# Let's see how it looks
qtable

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [4]:
qtable.shape

(16, 4)

In [12]:
# Randomly choose an action
environment.action_space.sample()

3

◀️ LEFT = 0

🔽 DOWN = 1

▶️ RIGHT = 2

🔼 UP = 3

In [6]:
# Move 2 to the right
environment.step(2)

# Check if the agent moved
environment.render()


In [8]:
action = environment.action_space.sample()

# 2. Implement this action and move the agent in the desired direction
new_state, reward, done, _, info = environment.step(action)

# Display the results (reward and map)
environment.render()

print(f'Reward = {reward}')

Reward = 0.0


In [9]:
environment.render()  # WHY IS IT NOT RENDERING?